# Project 3: Smart Beta Portfolio and Portfolio Optimization

## Overview


Smart beta has a broad meaning, but we can say in practice that when we use the universe of stocks from an index, and then apply some weighting scheme other than market cap weighting, it can be considered a type of smart beta fund.  A Smart Beta portfolio generally gives investors exposure or "beta" to one or more types of market characteristics (or factors) that are believed to predict prices while giving investors a diversified broad exposure to a particular market. Smart Beta portfolios generally target momentum, earnings quality, low volatility, and dividends or some combination. Smart Beta Portfolios are generally rebalanced infrequently and follow relatively simple rules or algorithms that are passively managed.  Model changes to these types of funds are also rare requiring prospectus filings with US Security and Exchange Commission in the case of US focused mutual funds or ETFs.. Smart Beta portfolios are generally long-only, they do not short stocks.

In contrast, a purely alpha-focused quantitative fund may use multiple models or algorithms to create a portfolio. The portfolio manager retains discretion in upgrading or changing the types of models and how often to rebalance the portfolio in attempt to maximize performance in comparison to a stock benchmark.  Managers may have discretion to short stocks in portfolios.

Imagine you're a portfolio manager, and wish to try out some different portfolio weighting methods.

One way to design portfolio is to look at certain accounting measures (fundamentals) that, based on past trends, indicate stocks that produce better results.  


For instance, you may start with a hypothesis that dividend-issuing stocks tend to perform better than stocks that do not. This may not always be true of all companies; for instance, Apple does not issue dividends, but has had good historical performance.  The hypothesis about dividend-paying stocks may go something like this: 

Companies that regularly issue dividends may also be more prudent in allocating their available cash, and may indicate that they are more conscious of prioritizing shareholder interests.  For example, a CEO may decide to reinvest cash into pet projects that produce low returns.  Or, the CEO may do some analysis, identify that reinvesting within the company produces lower returns compared to a diversified portfolio, and so decide that shareholders would be better served if they were given the cash (in the form of dividends).  So according to this hypothesis, dividends may be both a proxy for how the company is doing (in terms of earnings and cash flow), but also a signal that the company acts in the best interest of its shareholders.  Of course, it's important to test whether this works in practice.


You may also have another hypothesis, with which you wish to design a portfolio that can then be made into an ETF.  You may find that investors may wish to invest in passive beta funds, but wish to have less risk exposure (less volatility) in their investments.  The goal of having a low volatility fund that still produces returns similar to an index may be appealing to investors who have a shorter investment time horizon, and so are more risk averse.

So the objective of your proposed portfolio is to design a portfolio that closely tracks an index, while also minimizing the portfolio variance.  Also, if this portfolio can match the returns of the index with less volatility, then it has a higher risk-adjusted return (same return, lower volatility).

Smart Beta ETFs can be designed with both of these two general methods (among others): alternative weighting and minimum volatility ETF.


## Instructions
Each problem consists of a function to implement and instructions on how to implement the function.  The parts of the function that need to be implemented are marked with a `# TODO` comment. After implementing the function, run the cell to test it against the unit tests we've provided. For each problem, we provide one or more unit tests from our `project_tests` package. These unit tests won't tell you if your answer is correct, but will warn you of any major errors. Your code will be checked for the correct solution when you submit it to Udacity.

## Packages
When you implement the functions, you'll only need to you use the packages you've used in the classroom, like [Pandas](https://pandas.pydata.org/) and [Numpy](http://www.numpy.org/). These packages will be imported for you. We recommend you don't add any import statements, otherwise the grader might not be able to run your code.

The other packages that we're importing are `helper`, `project_helper`, and `project_tests`. These are custom packages built to help you solve the problems.  The `helper` and `project_helper` module contains utility functions and graph functions. The `project_tests` contains the unit tests for all the problems.
### Install Packages

In [1]:
import sys
!{sys.executable} -m pip install -r requirements.txt

You are using pip version 9.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


### Load Packages

In [1]:
import pandas as pd
import numpy as np
import helper
import project_helper
import project_tests

## Market Data
### Load Data
For this universe of stocks, we'll be selecting large dollar volume stocks. We're using this universe, since it is highly liquid.

In [2]:
df = pd.read_csv('eod-quotemedia.csv')
print(df.info())
percent_top_dollar = 0.2
high_volume_symbols = project_helper.large_dollar_volume_stocks(df, 'adj_close', 'adj_volume', percent_top_dollar)
df = df[df['ticker'].isin(high_volume_symbols)]

close = df.reset_index().pivot(index='date', columns='ticker', values='adj_close')
volume = df.reset_index().pivot(index='date', columns='ticker', values='adj_volume')
dividends = df.reset_index().pivot(index='date', columns='ticker', values='dividends')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 490737 entries, 0 to 490736
Data columns (total 6 columns):
Unnamed: 0    490737 non-null int64
date          490737 non-null object
ticker        490737 non-null object
adj_close     490737 non-null float64
adj_volume    490737 non-null float64
dividends     490737 non-null float64
dtypes: float64(3), int64(1), object(2)
memory usage: 22.5+ MB
None


### View Data
To see what one of these 2-d matrices looks like, let's take a look at the closing prices matrix.

In [3]:
project_helper.print_dataframe(close)

# Part 1: Smart Beta Portfolio
In Part 1 of this project, you'll build a portfolio using dividend yield to choose the portfolio weights. A portfolio such as this could be incorporated into a smart beta ETF.  You'll compare this portfolio to a market cap weighted index to see how well it performs. 

Note that in practice, you'll probably get the index weights from a data vendor (such as companies that create indices, like MSCI, FTSE, Standard and Poor's), but for this exercise we will simulate a market cap weighted index.

## Index Weights
The index we'll be using is based on large dollar volume stocks. Implement `generate_dollar_volume_weights` to generate the weights for this index. For each date, generate the weights based on dollar volume traded for that date. For example, assume the following is close prices and volume data:
```
                 Prices
               A         B         ...
2013-07-08     2         2         ...
2013-07-09     5         6         ...
2013-07-10     1         2         ...
2013-07-11     6         5         ...
...            ...       ...       ...

                 Volume
               A         B         ...
2013-07-08     100       340       ...
2013-07-09     240       220       ...
2013-07-10     120       500       ...
2013-07-11     10        100       ...
...            ...       ...       ...
```
The weights created from the function `generate_dollar_volume_weights` should be the following:
```
               A         B         ...
2013-07-08     0.126..   0.194..   ...
2013-07-09     0.759..   0.377..   ...
2013-07-10     0.075..   0.285..   ...
2013-07-11     0.037..   0.142..   ...
...            ...       ...       ...
```

In [24]:
def generate_dollar_volume_weights(close, volume):
    """
    Generate dollar volume weights.

    Parameters
    ----------
    close : DataFrame
        Close price for each ticker and date
    volume : str
        Volume for each ticker and date

    Returns
    -------
    dollar_volume_weights : DataFrame
        The dollar volume weights for each ticker and date
    """
    assert close.index.equals(volume.index)
    assert close.columns.equals(volume.columns)
    
    #TODO: Implement function
    #close.value*volume.value/row_sum(close.value*volume.value)
#     print(close)
#     print(volume)
    dollar_volume = close*volume
    dollar_volume_sum = dollar_volume.sum(axis =1 )
    result = (dollar_volume.div(dollar_volume_sum,axis=0) )
    return result

project_tests.test_generate_dollar_volume_weights(generate_dollar_volume_weights)

Tests Passed


### View Data
Let's generate the index weights using `generate_dollar_volume_weights` and view them using a heatmap.

In [72]:
index_weights = generate_dollar_volume_weights(close, volume)
print(index_weights)
project_helper.plot_weights(index_weights, 'Index Weights')

ticker            AAL       AAPL       ABBV        ABT        AGN        AIG  \
date                                                                           
2013-07-01 0.00458693 0.11767248 0.00363163 0.00452034 0.00390809 0.01193725   
2013-07-02 0.00363550 0.13639979 0.00330692 0.00387337 0.00266703 0.01524297   
2013-07-03 0.00466860 0.13526072 0.00352350 0.01058904 0.00348543 0.01245591   
2013-07-05 0.00319288 0.11370932 0.00301426 0.00447694 0.00289257 0.01236056   
2013-07-08 0.00267775 0.09203767 0.00334354 0.00549484 0.00250670 0.00909724   
2013-07-09 0.00381208 0.10082010 0.00288745 0.00669273 0.00230203 0.01324442   
2013-07-10 0.00447257 0.09036346 0.00267033 0.00667761 0.00329537 0.01009668   
2013-07-11 0.00279390 0.08937189 0.00297546 0.00567649 0.00274211 0.01334972   
2013-07-12 0.00324340 0.07013009 0.00393849 0.00358572 0.00272487 0.01016838   
2013-07-15 0.00343331 0.07690605 0.00282922 0.00475185 0.00337597 0.00968997   
2013-07-16 0.00400739 0.06808705 0.00310

## Portfolio Weights
Now that we have the index weights, let's choose the portfolio weights based on dividend. You would normally calculate the weights based on trailing dividend yield, but we'll simplify this by just calculating the total dividend yield over time.

Implement `calculate_dividend_weights` to return the weights for each stock based on its total dividend yield over time. This is similar to generating the weight for the index, but it's using dividend data instead.
For example, assume the following is `dividends` data:
```
                 Prices
               A         B
2013-07-08     0         0
2013-07-09     0         1
2013-07-10     0.5       0
2013-07-11     0         0
2013-07-12     2         0
...            ...       ...
```
The weights created from the function `calculate_dividend_weights` should be the following:
```
               A         B
2013-07-08     NaN       NaN
2013-07-09     0         1
2013-07-10     0.333..   0.666..
2013-07-11     0.333..   0.666..
2013-07-12     0.714..   0.285..
...            ...       ...
```

In [33]:
def calculate_dividend_weights(dividends):
    """
    Calculate dividend weights.

    Parameters
    ----------
    dividends : DataFrame
        Dividend for each stock and date

    Returns
    -------
    dividend_weights : DataFrame
        Weights for each stock and date
    """
    #TODO: Implement function
    #列累积和/行汇总
    dividends_cum = dividends.cumsum()
    dividends_cum_sum = dividends_cum.sum(axis =1 )
    result = (dividends_cum.div(dividends_cum_sum,axis=0) ) 
    return result

project_tests.test_calculate_dividend_weights(calculate_dividend_weights)

Tests Passed


### View Data
Just like the index weights, let's generate the ETF weights and view them using a heatmap.

In [73]:
etf_weights = calculate_dividend_weights(dividends)
print(etf_weights)
project_helper.plot_weights(etf_weights, 'ETF Weights')

ticker            AAL       AAPL       ABBV        ABT        AGN        AIG  \
date                                                                           
2013-07-01 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000   
2013-07-02 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000   
2013-07-03 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000   
2013-07-05 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000   
2013-07-08 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000   
2013-07-09 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000   
2013-07-10 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000   
2013-07-11 0.00000000 0.00000000 0.04801921 0.01680672 0.00000000 0.00000000   
2013-07-12 0.00000000 0.00000000 0.04801921 0.01680672 0.00000000 0.00000000   
2013-07-15 0.00000000 0.00000000 0.04696214 0.01643675 0.00000000 0.00000000   
2013-07-16 0.00000000 0.00000000 0.04696

## Returns
Implement `generate_returns` to generate returns data for all the stocks and dates from price data. You might notice we're implementing returns and not log returns. Since we're not dealing with volatility, we don't have to use log returns.

In [71]:
def generate_returns(prices):
    """
    Generate returns for ticker and date.

    Parameters
    ----------
    prices : DataFrame
        Price for each ticker and date

    Returns
    -------
    returns : Dataframe
        The returns for each ticker and date
    """
    #TODO: Implement function
    print (prices)
    prices_shift1 = prices.shift(1)
    res = ((prices-prices_shift1)/prices_shift1)
    print(res)
    return res
    
project_tests.test_generate_returns(generate_returns)

                   EJG        ENXX         PYP
2015-02-17 35.44110000 34.17990000 34.02230000
2015-02-18 92.11310000 91.05430000 90.95720000
2015-02-19 57.97080000 57.78140000 58.19820000
2015-02-20 34.17050000 92.45300000 58.51070000
                   EJG        ENXX         PYP
2015-02-17         nan         nan         nan
2015-02-18  1.59904743  1.66397210  1.67345829
2015-02-19 -0.37065629 -0.36541822 -0.36015840
2015-02-20 -0.41055669  0.60004777  0.00536958
Tests Passed


### View Data
Let's generate the closing returns using `generate_returns` and view them using a heatmap.

In [74]:
returns = generate_returns(close)
print(returns)
project_helper.plot_returns(returns, 'Close Returns')

ticker             AAL         AAPL        ABBV         ABT          AGN  \
date                                                                       
2013-07-01 16.17609308  53.10917319 34.92447839 31.42538772 122.62751990   
2013-07-02 15.81983388  54.31224742 35.42807578 31.27288084 121.05361758   
2013-07-03 16.12794994  54.61204262 35.44486235 30.72565028 121.21003024   
2013-07-05 16.21460758  54.17338125 35.85613355 31.32670680 123.53666845   
2013-07-08 16.31089385  53.86579916 36.66188936 31.76628544 123.65397794   
2013-07-09 16.71529618  54.81320389 36.35973093 31.16522893 122.89146626   
2013-07-10 16.53235227  54.60295791 36.85493502 31.16522893 121.28823656   
2013-07-11 16.72492481  55.45406479 37.08155384 31.85599537 122.19738511   
2013-07-12 16.90786872  55.35309481 38.15724076 31.81096287 123.35092842   
2013-07-15 17.10044125  55.47379158 37.79303181 31.95506689 122.37334934   
2013-07-16 17.28338516  55.83133953 37.10696377 32.15320992 120.08581429   
2013-07-17 1

## Weighted Returns
With the returns of each stock computed, we can use it to compute the returns for an index or ETF. Implement `generate_weighted_returns` to create weighted returns using the returns and weights.

In [41]:
def generate_weighted_returns(returns, weights):
    """
    Generate weighted returns.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date
    weights : DataFrame
        Weights for each ticker and date

    Returns
    -------
    weighted_returns : DataFrame
        Weighted returns for each ticker and date
    """
    assert returns.index.equals(weights.index)
    assert returns.columns.equals(weights.columns)
    
    #TODO: Implement function

    return returns*weights

project_tests.test_generate_weighted_returns(generate_weighted_returns)

Tests Passed


### View Data
Let's generate the ETF and index returns using `generate_weighted_returns` and view them using a heatmap.

In [75]:
index_weighted_returns = generate_weighted_returns(returns, index_weights)
etf_weighted_returns = generate_weighted_returns(returns, etf_weights)
print (index_weighted_returns)
print (etf_weighted_returns)
project_helper.plot_returns(index_weighted_returns, 'Index Returns')
project_helper.plot_returns(etf_weighted_returns, 'ETF Returns')

ticker             AAL        AAPL        ABBV         ABT         AGN  \
date                                                                     
2013-07-01         nan         nan         nan         nan         nan   
2013-07-02 -0.00008007  0.00308984  0.00004768 -0.00001880 -0.00003423   
2013-07-03  0.00009093  0.00074662  0.00000167 -0.00018529  0.00000450   
2013-07-05  0.00001716 -0.00091335  0.00003497  0.00008758  0.00005552   
2013-07-08  0.00001590 -0.00052257  0.00007514  0.00007710  0.00000238   
2013-07-09  0.00009451  0.00177325 -0.00002380 -0.00012663 -0.00001420   
2013-07-10 -0.00004895 -0.00034661  0.00003637  0.00000000 -0.00004299   
2013-07-11  0.00003254  0.00139306  0.00001830  0.00012582  0.00002055   
2013-07-12  0.00003548 -0.00012769  0.00011425 -0.00000507  0.00002572   
2013-07-15  0.00003910  0.00016769 -0.00002700  0.00002153 -0.00002676   
2013-07-16  0.00004287  0.00043884 -0.00005637  0.00003814 -0.00005587   
2013-07-17  0.00018322  0.00001630  0.

## ？Cumulative Returns
To compare performance between the ETF and Index, we're going to calculate the tracking error. Before we do that, we first need to calculate the index and ETF comulative returns. Implement `calculate_cumulative_returns` to calculate the cumulative returns over time given the returns.

In [135]:
def calculate_cumulative_returns(returns):
    """
    Calculate cumulative returns.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date

    Returns
    -------
    cumulative_returns : Pandas Series
        Cumulative returns for each date
    """
    #TODO: Implement function  r=(1+r1)∗(1+r2)∗(1+r3)∗(1+r4) −1
    print(returns.shape)
    print(returns)
    returns = (returns + 1).cumprod()-1
    print(returns)
    #returns.cumsum(axis =1).iloc[returns.shape[1]-1]/returns.shape[1]
    return  returns.sum(axis=1)+1

project_tests.test_calculate_cumulative_returns(calculate_cumulative_returns)

(4, 3)
                   WYI        IGMB         NDC
2016-08-14         nan         nan         nan
2016-08-15  1.59904743  1.66397210  1.67345829
2016-08-16 -0.37065629 -0.36541822 -0.36015840
2016-08-17 -0.41055669  0.60004777  0.00536958
                   WYI       IGMB        NDC
2016-08-14         nan        nan        nan
2016-08-15  1.59904743 1.66397210 1.67345829
2016-08-16  0.63569415 0.69050816 0.71058983
2016-08-17 -0.03585102 1.70489381 0.71977498


AssertionError: Wrong value for calculate_cumulative_returns.

INPUT returns:
                   WYI        IGMB         NDC
2016-08-14         nan         nan         nan
2016-08-15  1.59904743  1.66397210  1.67345829
2016-08-16 -0.37065629 -0.36541822 -0.36015840
2016-08-17 -0.41055669  0.60004777  0.00536958

OUTPUT cumulative_returns:
2016-08-14          nan
2016-08-15   5.93647782
2016-08-16   3.03679214
2016-08-17   3.38881776
dtype: float64

EXPECTED OUTPUT FOR cumulative_returns:
2016-08-14           nan
2016-08-15    5.93647782
2016-08-16   -0.57128454
2016-08-17   -0.68260542
dtype: float64


### View Data
Let's generate the ETF and index cumulative returns using `calculate_cumulative_returns` and compare the two.

In [ ]:
index_weighted_cumulative_returns = calculate_cumulative_returns(index_weighted_returns)
etf_weighted_cumulative_returns = calculate_cumulative_returns(etf_weighted_returns)
project_helper.plot_benchmark_returns(index_weighted_cumulative_returns, etf_weighted_cumulative_returns, 'Smart Beta ETF vs Index')

## Tracking Error
In order to check the performance of the smart beta portfolio, we can calculate the annualized tracking error against the index. Implement `tracking_error` to return the tracking error between the ETF and benchmark.

For reference, we'll be using the following annualized tracking error function:
$$ TE = \sqrt{252} * SampleStdev(r_p - r_b) $$

Where $ r_p $ is the portfolio/ETF returns and $ r_b $ is the benchmark returns.

_Note: When calculating the sample standard deviation, the delta degrees of freedom is 1, which is the also the default value._

In [87]:
import math
def tracking_error(benchmark_returns_by_date, etf_returns_by_date):
    """
    Calculate the tracking error.

    Parameters
    ----------
    benchmark_returns_by_date : Pandas Series
        The benchmark returns for each date
    etf_returns_by_date : Pandas Series
        The ETF returns for each date

    Returns
    -------
    tracking_error : float
        The tracking error
    """
    assert benchmark_returns_by_date.index.equals(etf_returns_by_date.index)
     #TODO: Implement function
    res =(etf_returns_by_date-benchmark_returns_by_date).std()*math.sqrt(252)
    print(res)
    return res

project_tests.test_tracking_error(tracking_error)

16.526243197132676
Tests Passed


### View Data
Let's generate the tracking error using `tracking_error`.

In [88]:
smart_beta_tracking_error = tracking_error(np.sum(index_weighted_returns, 1), np.sum(etf_weighted_returns, 1))
print('Smart Beta Tracking Error: {}'.format(smart_beta_tracking_error))

0.1020761483200753
Smart Beta Tracking Error: 0.1020761483200753


# Part 2: Portfolio Optimization

Now, let's create a second portfolio.  We'll still reuse the market cap weighted index, but this will be independent of the dividend-weighted portfolio that we created in part 1.

We want to both minimize the portfolio variance and also want to closely track a market cap weighted index.  In other words, we're trying to minimize the distance between the weights of our portfolio and the weights of the index.

$Minimize \left [ \sigma^2_p + \lambda \sqrt{\sum_{1}^{m}(weight_i - indexWeight_i)^2} \right  ]$ where $m$ is the number of stocks in the portfolio, and $\lambda$ is a scaling factor that you can choose.

Why are we doing this? One way that investors evaluate a fund is by how well it tracks its index. The fund is still expected to deviate from the index within a certain range in order to improve fund performance.  A way for a fund to track the performance of its benchmark is by keeping its asset weights similar to the weights of the index.  We’d expect that if the fund has the same stocks as the benchmark, and also the same weights for each stock as the benchmark, the fund would yield about the same returns as the benchmark. By minimizing a linear combination of both the portfolio risk and distance between portfolio and benchmark weights, we attempt to balance the desire to minimize portfolio variance with the goal of tracking the index.


## Covariance
Implement `get_covariance_returns` to calculate the covariance of the `returns`. We'll use this to calculate the portfolio variance.

If we have $m$ stock series, the covariance matrix is an $m \times m$ matrix containing the covariance between each pair of stocks.  We can use [`Numpy.cov`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.cov.html) to get the covariance.  We give it a 2D array in which each row is a stock series, and each column is an observation at the same period of time. For any `NaN` values, you can replace them with zeros using the [`DataFrame.fillna`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.fillna.html) function.

The covariance matrix $\mathbf{P} = 
\begin{bmatrix}
\sigma^2_{1,1} & ... & \sigma^2_{1,m} \\ 
... & ... & ...\\
\sigma_{m,1} & ... & \sigma^2_{m,m}  \\
\end{bmatrix}$

In [104]:
x = [-2.1, -1,  4.3]
y = [3,  1.1,  0.12]
X = np.stack((x, y), axis=0)
print(X)
print(np.cov(X))
print(np.cov(x, y))
print(np.cov(x))

[[-2.1  -1.    4.3 ]
 [ 3.    1.1   0.12]]
[[11.71       -4.286     ]
 [-4.286       2.14413333]]
[[11.71       -4.286     ]
 [-4.286       2.14413333]]
11.709999999999999


In [152]:
def get_covariance_returns(returns):
    """
    Calculate covariance matrices.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date

    Returns
    -------
    returns_covariance  : 2 dimensional Ndarray
        The covariance of the returns
    """
    #TODO: Implement function
    returns = returns.fillna(0)
   # print(returns.head())
    cov_res = np.cov(returns.T)
    #print(cov_res.head())
    return cov_res

project_tests.test_get_covariance_returns(get_covariance_returns)

Tests Passed


### View Data
Let's look at the covariance generated from `get_covariance_returns`.

In [105]:
covariance_returns = get_covariance_returns(returns)
covariance_returns = pd.DataFrame(covariance_returns, returns.columns, returns.columns)

covariance_returns_correlation = np.linalg.inv(np.diag(np.sqrt(np.diag(covariance_returns))))
covariance_returns_correlation = pd.DataFrame(
    covariance_returns_correlation.dot(covariance_returns).dot(covariance_returns_correlation),
    covariance_returns.index,
    covariance_returns.columns)

project_helper.plot_covariance_returns_correlation(
    covariance_returns_correlation,
    'Covariance Returns Correlation Matrix')

ticker             AAL        AAPL        ABBV         ABT         AGN  \
date                                                                     
2013-07-01  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000   
2013-07-02 -0.02202381  0.02265285  0.01441961 -0.00485298 -0.01283482   
2013-07-03  0.01947657  0.00551985  0.00047382 -0.01749857  0.00129209   
2013-07-05  0.00537313 -0.00803232  0.01160313  0.01956204  0.01919510   
2013-07-08  0.00593824 -0.00567773  0.02247191  0.01403207  0.00094959   
2013-07-09  0.02479339  0.01758824 -0.00824176 -0.01892121 -0.00616650   
2013-07-10 -0.01094470 -0.00383568  0.01361958  0.00000000 -0.01304590   
2013-07-11  0.01164822  0.01558719  0.00614894  0.02216465  0.00749577   
2013-07-12  0.01093840 -0.00182079  0.02900868 -0.00141363  0.00944000   
2013-07-15  0.01138952  0.00218049 -0.00954495  0.00453001 -0.00792519   
2013-07-16  0.01069820  0.00644535 -0.01815329  0.00620068 -0.01869308   
2013-07-17  0.02785515  0.00026732  0.

### portfolio variance
We can write the portfolio variance $\sigma^2_p = \mathbf{x^T} \mathbf{P} \mathbf{x}$

Recall that the $\mathbf{x^T} \mathbf{P} \mathbf{x}$ is called the quadratic form.
We can use the cvxpy function `quad_form(x,P)` to get the quadratic form.

### Distance from index weights
We want portfolio weights that track the index closely.  So we want to minimize the distance between them.
Recall from the Pythagorean theorem that you can get the distance between two points in an x,y plane by adding the square of the x and y distances and taking the square root.  Extending this to any number of dimensions is called the L2 norm.  So: $\sqrt{\sum_{1}^{n}(weight_i - indexWeight_i)^2}$  Can also be written as $\left \| \mathbf{x} - \mathbf{index} \right \|_2$.  There's a cvxpy function called [norm()](https://www.cvxpy.org/api_reference/cvxpy.atoms.other_atoms.html#norm)
`norm(x, p=2, axis=None)`.  The default is already set to find an L2 norm, so you would pass in one argument, which is the difference between your portfolio weights and the index weights.

### objective function
We want to minimize both the portfolio variance and the distance of the portfolio weights from the index weights.
We also want to choose a `scale` constant, which is $\lambda$ in the expression. 

$\mathbf{x^T} \mathbf{P} \mathbf{x} + \lambda \left \| \mathbf{x} - \mathbf{index} \right \|_2$


This lets us choose how much priority we give to minimizing the difference from the index, relative to minimizing the variance of the portfolio.  If you choose a higher value for `scale` ($\lambda$).

We can find the objective function using cvxpy `objective = cvx.Minimize()`.  Can you guess what to pass into this function?



### constraints
We can also define our constraints in a list.  For example, you'd want the weights to sum to one. So $\sum_{1}^{n}x = 1$.  You may also need to go long only, which means no shorting, so no negative weights.  So $x_i >0 $ for all $i$. you could save a variable as `[x >= 0, sum(x) == 1]`, where x was created using `cvx.Variable()`.

### optimization
So now that we have our objective function and constraints, we can solve for the values of $\mathbf{x}$.
cvxpy has the constructor `Problem(objective, constraints)`, which returns a `Problem` object.

The `Problem` object has a function solve(), which returns the minimum of the solution.  In this case, this is the minimum variance of the portfolio.

It also updates the vector $\mathbf{x}$.

We can check out the values of $x_A$ and $x_B$ that gave the minimum portfolio variance by using `x.value`

In [216]:
import cvxpy as cvx

def get_optimal_weights(covariance_returns, index_weights, scale=2.0):
    """
    Find the optimal weights.

    Parameters
    ----------
    covariance_returns : 2 dimensional Ndarray
        The covariance of the returns
    index_weights : Pandas Series
        Index weights for all tickers at a period in time
    scale : int
        The penalty factor for weights the deviate from the index 
    Returns
    -------
    x : 1 dimensional Ndarray
        The solution for x
    """
    assert len(covariance_returns.shape) == 2
    assert len(index_weights.shape) == 1
    assert covariance_returns.shape[0] == covariance_returns.shape[1]  == index_weights.shape[0]

    #TODO: Implement function
    # number of stocks m is number of rows of returns, and also number of index weights
    m = len(covariance_returns)
    
#     #covariance matrix of returns
    #cov = np.cov(covariance_returns)
    cov = covariance_returns
    
#     # x variables (to be found with optimization)
    x = cvx.Variable(m) 
    
#     #portfolio variance, in quadratic form
    portfolio_variance = cvx.quad_form(x,cov)
    
#     # euclidean distance (L2 norm) between portfolio and index weights
    distance_to_index = cvx.norm(x - index_weights)
    
#     #objective function
    objective =  cvx.Minimize(portfolio_variance + scale * distance_to_index)
    
#     #constraints
    constraints = [x >= 0, sum(x) == 1]

#     #use cvxpy to solve the objective
    cvx.Problem(objective, constraints).solve()
#     #retrieve the weights of the optimized portfolio
    x_values = x.value
 #   print (covariance_returns)
  #  print (index_weights)
    #print(x_values)
    return x_values


project_tests.test_get_optimal_weights(get_optimal_weights)

Tests Passed


## Optimized Portfolio
Using the `get_optimal_weights` function, let's generate the optimal ETF weights without rebalanceing. We can do this by feeding in the covariance of the entire history of data. We also need to feed in a set of index weights. We'll go with the average weights of the index over time.

In [117]:
raw_optimal_single_rebalance_etf_weights = get_optimal_weights(covariance_returns.values, index_weights.iloc[-1])
optimal_single_rebalance_etf_weights = pd.DataFrame(
    np.tile(raw_optimal_single_rebalance_etf_weights, (len(returns.index), 1)),
    returns.index,
    returns.columns)

[[5.31791451e-04 7.21251856e-05 9.55847963e-05 8.96491142e-05
  1.15101755e-04 9.46783638e-05 1.35121643e-04 1.17865938e-04
  9.16401993e-05 8.41973376e-05 1.58239743e-04 1.02063056e-04
  1.00626382e-04 1.43691378e-04 1.35720980e-04 7.48139675e-05
  1.45713010e-04 9.10144040e-05 1.03978245e-04 1.47556123e-04
  8.62962337e-05 6.67031628e-05 7.53241762e-05 6.95054622e-05
  5.60037479e-05 1.35809580e-04 8.22672743e-05 6.42316752e-05
  5.32743654e-05 3.61610193e-04 8.73032318e-05 1.06785280e-04
  7.34901165e-05 8.88186161e-05 1.18621080e-04 1.25859843e-04
  1.32572381e-04 1.07931561e-04 9.52116796e-05 7.35449702e-05
  1.22133476e-04 1.23011264e-04 8.68982283e-05 9.57539624e-05
  1.28187931e-04 8.86011950e-05 8.25499308e-05 9.27496878e-05
  6.24019398e-05 1.03133183e-04 5.21107784e-05 1.18927344e-04
  9.14518031e-05 3.50902820e-05 6.78741141e-05 6.61823284e-05
  9.15333188e-05 1.17873327e-04 5.25366352e-05 7.32022220e-05
  8.25646805e-05 3.60958694e-05 6.86508571e-05 7.03767424e-05
  1.5535

In [124]:
print((raw_optimal_single_rebalance_etf_weights))
print(optimal_single_rebalance_etf_weights)

[0.00505383 0.04996914 0.00507628 0.00344529 0.00631443 0.00723481
 0.00628914 0.00690689 0.05022972 0.00367206 0.00923578 0.00420389
 0.00659224 0.03065369 0.0057022  0.00386121 0.0159118  0.00755899
 0.00230404 0.00784448 0.00719021 0.01067014 0.00471339 0.00600604
 0.01150549 0.00522686 0.01085341 0.00530558 0.00845056 0.00402521
 0.0081877  0.00579648 0.00573072 0.00533293 0.00660151 0.04082251
 0.00271105 0.004656   0.00335664 0.01048722 0.00708355 0.00395319
 0.02907164 0.03255119 0.0105735  0.00460467 0.00836054 0.00353609
 0.00806711 0.01229801 0.01038399 0.02300489 0.00370103 0.00620664
 0.00700668 0.00450263 0.00544879 0.00660128 0.00573769 0.0037471
 0.00505419 0.00563285 0.01461977 0.00793407 0.00560465 0.0250028
 0.03350294 0.01192162 0.04150111 0.040325   0.00993157 0.00412432
 0.00568072 0.00669019 0.00783857 0.00553949 0.00366138 0.00714256
 0.00952813 0.00709645 0.00780563 0.01234318 0.00308681 0.00712188
 0.00539759 0.00398087 0.00669447 0.00567573 0.00357363 0.005318

With our ETF weights built, let's compare it to the index. Run the next cell to calculate the ETF returns and compare it to the index returns.

In [125]:
optim_etf_returns = generate_weighted_returns(returns, optimal_single_rebalance_etf_weights)
optim_etf_cumulative_returns = calculate_cumulative_returns(optim_etf_returns)
project_helper.plot_benchmark_returns(index_weighted_cumulative_returns, optim_etf_cumulative_returns, 'Optimized ETF vs Index')

optim_etf_tracking_error = tracking_error(np.sum(index_weighted_returns, 1), np.sum(optim_etf_returns, 1))
print('Optimized ETF Tracking Error: {}'.format(optim_etf_tracking_error))

(1009, 99)
ticker             AAL        AAPL        ABBV         ABT         AGN  \
date                                                                     
2013-07-01         nan         nan         nan         nan         nan   
2013-07-02 -0.00011130  0.00113194  0.00007320 -0.00001672 -0.00008104   
2013-07-03  0.00009843  0.00027582  0.00000241 -0.00006029  0.00000816   
2013-07-05  0.00002715 -0.00040137  0.00005890  0.00006740  0.00012121   
2013-07-08  0.00003001 -0.00028371  0.00011407  0.00004834  0.00000600   
2013-07-09  0.00012530  0.00087887 -0.00004184 -0.00006519 -0.00003894   
2013-07-10 -0.00005531 -0.00019167  0.00006914  0.00000000 -0.00008238   
2013-07-11  0.00005887  0.00077888  0.00003121  0.00007636  0.00004733   
2013-07-12  0.00005528 -0.00009098  0.00014726 -0.00000487  0.00005961   
2013-07-15  0.00005756  0.00010896 -0.00004845  0.00001561 -0.00005004   
2013-07-16  0.00005407  0.00032207 -0.00009215  0.00002136 -0.00011804   
2013-07-17  0.00014078  0.0

NameError: name 'index_weighted_cumulative_returns' is not defined

## Rebalance Portfolio Over Time
The single optimized ETF portfolio used the same weights for the entire history. This might not be the optimal weights for the entire period. Let's rebalance the portfolio over the same period instead of using the same weights. Implement `rebalance_portfolio` to rebalance a portfolio.

Reblance the portfolio every n number of days, which is given as `shift_size`. When rebalancing, you should look back a certain number of days of data in the past, denoted as `chunk_size`. Using this data, compute the optoimal weights using `get_optimal_weights` and `get_covariance_returns`.

In [142]:
get_covariance_returns??
#get_optimal_weights??

In [226]:
def rebalance_portfolio(returns, index_weights, shift_size, chunk_size):
    """
    Get weights for each rebalancing of the portfolio.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date
    index_weights : DataFrame
        Index weight for each ticker and date
    shift_size : int
        The number of days between each rebalance
    chunk_size : int
        The number of days to look in the past for rebalancing

    Returns
    -------
    all_rebalance_weights  : list of Ndarrays
        The ETF weights for each point they are rebalanced
    """
    assert returns.index.equals(index_weights.index)
    assert returns.columns.equals(index_weights.columns)
    assert shift_size > 0
    assert chunk_size >= 0
    
    #TODO: Implement function
    rebalanced_weights = []
    for i in range(chunk_size, len(returns), shift_size): 
            start_index = i-chunk_size
            returns_chunk = returns[start_index:i]
            index_chunk = index_weights[start_index:i].iloc[-1]
            returns_chunk_cov = get_covariance_returns(returns_chunk)
            weight = get_optimal_weights(returns_chunk_cov, index_chunk)
            rebalanced_weights.append(weight)
    return rebalanced_weights

###############test###############
#     #print(returns, index_weights, shift_size, chunk_size)
#     result_in = returns.iloc[0:4]
#     print(result_in)
#     result_four = get_covariance_returns(result_in)
#     print(result_four)
#     index_weights_in = index_weights.iloc[3]
#     print(index_weights_in)
#     x_values = get_optimal_weights(result_four, index_weights_in)
#     print(x_values)
#     return x_values

project_tests.test_rebalance_portfolio(rebalance_portfolio)

Tests Passed


Run the following cell to rebalance the portfolio using `rebalance_portfolio`.

In [227]:
chunk_size = 250
shift_size = 5
all_rebalance_weights = rebalance_portfolio(returns, index_weights, shift_size, chunk_size)

## Portfolio Turnover
With the portfolio rebalanced, we need to use a metric to measure the cost of rebalancing the portfolio. Implement `get_portfolio_turnover` to calculate the annual portfolio turnover. We'll be using the formulas used in the classroom:

$ AnnualizedTurnover =\frac{SumTotalTurnover}{NumberOfRebalanceEvents} * NumberofRebalanceEventsPerYear $

$ SumTotalTurnover =\sum_{t,n}{\left | x_{t,n} - x_{t+1,n} \right |} $ Where $ x_{t,n} $ are the weights at time $ t $ for equity $ n $.

$ SumTotalTurnover $ is just a different way of writing $ \sum \left | x_{t_1,n} - x_{t_2,n} \right | $

In [228]:
def get_portfolio_turnover(all_rebalance_weights, shift_size, rebalance_count, n_trading_days_in_year=252):
    """
    Calculage portfolio turnover.

    Parameters
    ----------
    all_rebalance_weights : list of Ndarrays
        The ETF weights for each point they are rebalanced
    shift_size : int
        The number of days between each rebalance
    rebalance_count : int
        Number of times the portfolio was rebalanced
    n_trading_days_in_year: int
        Number of trading days in a year

    Returns
    -------
    portfolio_turnover  : float
        The portfolio turnover
    """
    assert shift_size > 0
    assert rebalance_count > 0
    
    #TODO: Implement function
    
    return None

project_tests.test_get_portfolio_turnover(get_portfolio_turnover)

AssertionError: Wrong type for output portfolio_turnover. Got <class 'NoneType'>, expected <class 'float'>

Run the following cell to get the portfolio turnover from  `get_portfolio turnover`.

In [ ]:
print(get_portfolio_turnover(all_rebalance_weights, shift_size, len(all_rebalance_weights) - 1))

That's it! You've built a smart beta portfolio in part 1 and did portfolio optimization in part 2. You can now submit your project.

## Submission
Now that you're done with the project, it's time to submit it. Click the submit button in the bottom right. One of our reviewers will give you feedback on your project with a pass or not passed grade. You can continue to the next section while you wait for feedback.